In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, re's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# SmartInvoice Agent — Capstone Project

## Overview
SmartInvoice Agent is an **AI-powered multi-agent system** designed to help users manage invoices efficiently. 
It allows users to **view invoices, check payment status, create support tickets, and add new invoices dynamically**. 
The agent also keeps a **log of all actions** and provides a **summary report**, making it suitable for enterprise or personal use.

## Features
1. **Invoice Management**
   - View details of invoices (customer, amount, status)
   - Check payment status
   - Add new invoices dynamically

2. **Ticket / Support Agent**
   - Automatically creates support tickets if there is an issue with an invoice
   - Tracks tickets and their statuses

3. **Memory & Session**
   - Remembers the **last invoice** interacted with, simplifying follow-up queries

4. **Observability & Logging**
   - Logs all actions like invoice views, payment checks, ticket creation
   - Provides a **summary of invoices and tickets** for easy review

5. **Interactive Demo**
   - Users can interact with the agent via a **chat-like interface**
   - Example interactions included to demonstrate capabilities

6. **Export Functionality**
   - Exports invoices and tickets to CSV files for reporting or backup

## How It Works
1. User inputs a command in the interactive demo or example interaction cell.
2. The **router function** directs the input to the appropriate agent:
   - `invoice_agent` → fetch and display invoice
   - `payment_agent` → show payment status
   - `ticket_agent` → create support ticket
   - `add_invoice` → add a new invoice dynamically
3. The agent performs the action, updates **memory**, **logs**, and **tickets** if applicable.
4. The user receives an immediate response, and all actions can be reviewed in the summary or logs.

## Example Commands
- "Show invoice 101"
- "What is its payment status?"
- "Help, invoice 103 is wrong"
- "Add invoice 105 Ali 3000 Pending"

---

**This notebook demonstrates a fully functional multi-agent invoice system ready for submission.**


In [ ]:
# ----------- MEMORY STORAGE -----------
# ----------- IMPORTS & MEMORY -----------
import datetime
import csv

memory = {}
logs = []
tickets = []


# ----------- INVOICE DATABASE -----------
invoices = {
    "101": {"customer": "Arfa", "amount": 2500, "status": "Paid"},
    "102": {"customer": "Sara", "amount": 1800, "status": "Pending"},
    "103": {"customer": "Ibrahim", "amount": 3000, "status": "Overdue"},
}

# ----------- FETCH FUNCTION -----------
def fetch_invoice(invoice_id):
    return invoices.get(invoice_id)

# ----------- AGENTS -----------

def invoice_agent(message, memory):
    invoice_id = ''.join(filter(str.isdigit, message))
    invoice = fetch_invoice(invoice_id)

    if invoice:
        return f"📄 Invoice {invoice_id}: Customer: {invoice['customer']}, Amount: {invoice['amount']}, Status: {invoice['status']}"
    else:
        return f"❗ Invoice {invoice_id} not found."

def payment_agent(message, memory):
    invoice_id = ''.join(filter(str.isdigit, message))
    invoice = fetch_invoice(invoice_id)

    if invoice:
        return f"💰 Payment Status for Invoice {invoice_id}: {invoice['status']}"
    else:
        return "❗ I cannot find that invoice."

# ----------- ROUTER -----------

def router(message, memory):
    m = message.lower()

    if "invoice" in m:
        return invoice_agent(message, memory)

    if "payment" in m or "status" in m:
        return payment_agent(message, memory)

    return "🤖 I don't understand your request."

# ----------- ASK FUNCTION -----------

def ask(user_msg):
    response = router(user_msg, memory)
    print("User:", user_msg)
    print("AI:", response)


In [ ]:
ask("I need invoice 101")
ask("What is its payment status?")
ask("Show invoice 103")


In [ ]:
# ----------- ADD NEW INVOICE FUNCTION -----------

def add_invoice(invoice_id, customer, amount, status):
    if invoice_id in invoices:
        return f"❗ Invoice {invoice_id} already exists."
    
    invoices[invoice_id] = {
        "customer": customer,
        "amount": amount,
        "status": status
    }
    return f"✅ Invoice {invoice_id} added successfully."


In [ ]:
print(add_invoice("104", "Fatima", 4200, "Pending"))
ask("Show invoice 104")


In [ ]:
# ----------- MEMORY UPDATE FOR LAST INVOICE -----------

def remember_invoice(invoice_id):
    memory['last_invoice'] = invoice_id

def get_last_invoice():
    return memory.get('last_invoice')


In [ ]:
def invoice_agent(message, memory):
    invoice_id = ''.join(filter(str.isdigit, message))
    invoice = fetch_invoice(invoice_id)

    if invoice:
        remember_invoice(invoice_id)  # Remember this invoice
        return f"📄 Invoice {invoice_id}: Customer: {invoice['customer']}, Amount: {invoice['amount']}, Status: {invoice['status']}"
    else:
        return f"❗ Invoice {invoice_id} not found."


In [ ]:
def payment_agent(message, memory):
    invoice_id = ''.join(filter(str.isdigit, message))
    
    # If user didn’t type invoice ID, use last one
    if not invoice_id:
        invoice_id = get_last_invoice()
        if not invoice_id:
            return "❗ Please specify an invoice number first."

    invoice = fetch_invoice(invoice_id)

    if invoice:
        remember_invoice(invoice_id)  # Remember again
        return f"💰 Payment Status for Invoice {invoice_id}: {invoice['status']}"
    else:
        return "❗ I cannot find that invoice."


In [ ]:
ask("Show invoice 102")
ask("What is its payment status?")
ask("What is the payment status?")


In [ ]:
# ----------- TICKET STORAGE -----------
tickets = []

# ----------- TICKET AGENT -----------
def ticket_agent(message, memory):
    last_invoice = get_last_invoice() or "N/A"
    
    ticket = {
        "issue": message,
        "invoice": last_invoice,
        "status": "open"
    }
    tickets.append(ticket)
    
    return f"✅ Support ticket created for invoice {last_invoice if last_invoice != 'N/A' else 'unknown'}.\nTicket ID: {len(tickets)}"


In [ ]:
def router(message, memory):
    m = message.lower()

    if "invoice" in m or "show" in m:
        return invoice_agent(message, memory)

    if "payment" in m or "status" in m:
        return payment_agent(message, memory)

    if "help" in m or "issue" in m or "problem" in m or "ticket" in m:
        return ticket_agent(message, memory)

    return "🤖 I don't understand your request."


In [ ]:
ask("I have a problem with invoice 103")
ask("Create a ticket")
ask("Help, invoice 102 is wrong")


In [ ]:
# ----------- INTERACTIVE DEMO -----------

def start_demo():
    print("🤖 Welcome to SmartInvoice Agent!")
    print("Type 'exit' to quit.\n")
    
    while True:
        user_msg = input("You: ")
        if user_msg.lower() == "exit":
            print("🤖 Goodbye!")
            break
        
        response = router(user_msg, memory)
        print("AI:", response)


In [ ]:
start_demo()


In [ ]:
# ----------- SUMMARY & EXPORT -----------

def summary():
    print("---- INVOICES ----")
    for inv_id, data in invoices.items():
        print(f"{inv_id}: {data}")
    print("\n---- TICKETS ----")
    for idx, t in enumerate(tickets, start=1):
        print(f"{idx}: Invoice {t['invoice']}, Issue: {t['issue']}, Status: {t['status']}")

def export_data():
    with open("invoices.csv", "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["invoice_id","customer","amount","status"])
        writer.writeheader()
        for inv_id, data in invoices.items():
            row = {"invoice_id": inv_id, **data}
            writer.writerow(row)

    with open("tickets.csv", "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["ticket_id","invoice","issue","status"])
        writer.writeheader()
        for idx, t in enumerate(tickets, start=1):
            row = {"ticket_id": idx, **t}
            writer.writerow(row)


In [ ]:
# ----------- LOGS & SUMMARY -----------

print("\n----- Action Logs -----")
for log in logs:
    print(log)

print("\n----- Invoice & Ticket Summary -----")
summary()


In [ ]:
# ----------- EXPORT CSV (Optional Bonus) -----------

export_data()
print("✅ Invoices and tickets exported to CSV files.")
